<a href="https://colab.research.google.com/github/up99/golf-sintez/blob/main/GolfHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/albivaltzew/golf_hack.git

Cloning into 'golf_hack'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 111 (delta 14), reused 10 (delta 10), pack-reused 85
Receiving objects: 100% (111/111), 634.27 KiB | 14.42 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd golf_hack

/content/golf_hack


In [ ]:
%cd data

/content/golf_hack/data


In [ ]:
%cd /content/golf_hack

/content/golf_hack


In [ ]:
!python /content/golf_hack/MobileNetV2.py

In [ ]:
!python3 test_video.py -p test_video.mp4

Preparing video: test_video.mp4
Traceback (most recent call last):
  File "/content/golf_hack/test_video.py", line 75, in <module>
    model = EventDetector(pretrain=True,
  File "/content/golf_hack/model.py", line 17, in __init__
    state_dict_mobilenet = torch.load('/content/drive/MyDrive/Hacks/golf/mobilenet_v2.pth.tar')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 815, in load
    return _legacy_load(opened_file, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 1043, in _legacy_load
    result = unpickler.load()
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 980, in persistent_load
    wrap_storage=restore_location(obj, location),
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 217, in default_restore_location
    result = fn(storage, location)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.

In [ ]:
from model import EventDetector

In [ ]:
from MobileNetV2 import MobileNetV2

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from MobileNetV2 import MobileNetV2


class EventDetector(nn.Module):
    def __init__(self, pretrain, width_mult, lstm_layers, lstm_hidden, bidirectional=True, dropout=True):
        super(EventDetector, self).__init__()
        self.width_mult = width_mult
        self.lstm_layers = lstm_layers
        self.lstm_hidden = lstm_hidden
        self.bidirectional = bidirectional
        self.dropout = dropout

        net = MobileNetV2(width_mult=width_mult)
        state_dict_mobilenet = torch.load('/content/drive/MyDrive/Hacks/golf/mobilenet_v2.pth.tar', map_location=torch.device('cpu'))
        if pretrain:
            net.load_state_dict(state_dict_mobilenet)

        self.cnn = nn.Sequential(*list(net.children())[0][:19])
        self.rnn = nn.LSTM(int(1280*width_mult if width_mult > 1.0 else 1280),
                           self.lstm_hidden, self.lstm_layers,
                           batch_first=True, bidirectional=bidirectional)
        if self.bidirectional:
            self.lin = nn.Linear(2*self.lstm_hidden, 9)
        else:
            self.lin = nn.Linear(self.lstm_hidden, 9)
        if self.dropout:
            self.drop = nn.Dropout(0.5)

    def init_hidden(self, batch_size):
        if self.bidirectional:
            return (Variable(torch.zeros(2*self.lstm_layers, batch_size, self.lstm_hidden), requires_grad=True),
                    Variable(torch.zeros(2*self.lstm_layers, batch_size, self.lstm_hidden), requires_grad=True))
        else:
            return (Variable(torch.zeros(self.lstm_layers, batch_size, self.lstm_hidden), requires_grad=True),
                    Variable(torch.zeros(self.lstm_layers, batch_size, self.lstm_hidden), requires_grad=True))

    def forward(self, x, lengths=None):
        batch_size, timesteps, C, H, W = x.size()
        self.hidden = self.init_hidden(batch_size)

        # CNN forward
        c_in = x.view(batch_size * timesteps, C, H, W)
        c_out = self.cnn(c_in)
        c_out = c_out.mean(3).mean(2)
        if self.dropout:
            c_out = self.drop(c_out)

        # LSTM forward
        r_in = c_out.view(batch_size, timesteps, -1)
        r_out, states = self.rnn(r_in, self.hidden)
        out = self.lin(r_out)
        out = out.view(batch_size*timesteps,9)

        return out





In [ ]:
model = EventDetector(pretrain=True,
                          width_mult=1.,
                          lstm_layers=1,
                          lstm_hidden=256,
                          bidirectional=True,
                          dropout=False)

In [ ]:
import torch

In [ ]:
pt_model_path = "/content/drive/MyDrive/Hacks/golf/mobilenet_v2.pth.tar"

In [ ]:
try:
    save_dict = torch.load('/content/drive/MyDrive/Hacks/golf/swingnet_1800.pth.tar', map_location=torch.device('cpu'))
except:
    print("Model weights not found. Download model weights and place in 'models' folder. See README for instructions")


In [ ]:
model.load_state_dict(save_dict['model_state_dict'])

<All keys matched successfully>

In [ ]:
model.eval()

EventDetector(
  (cnn): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3

In [ ]:
sample_input = torch.rand((1,100, 3, 160, 160)) #1280 - timesteps?

In [ ]:
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.6 MB/s eta 0:00:00


In [ ]:
onnx_model_path = "/content/drive/MyDrive/Hacks/golf/swing.onnx"

torch.onnx.export(
    model,                  # PyTorch Model
    sample_input,                    # Input tensor
    onnx_model_path,        # Output file (eg. 'output_model.onnx')
    opset_version=12,       # Operator support version
    # input_names=['input']   # Input tensor name (arbitary)
    # output_names=['output'] # Output tensor name (arbitary)
)

/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:4476: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
import onnx

In [ ]:
# Load the ONNX model
model = onnx.load("/content/drive/MyDrive/Hacks/golf/swing.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a Human readable representation of the graph
onnx.helper.printable_graph(model.graph)

"graph torch_jit (\n  %onnx::Reshape_0[FLOAT, 1x100x3x160x160]\n) initializers (\n  %lin.bias[FLOAT, 9]\n  %onnx::Conv_714[FLOAT, 32x3x3x3]\n  %onnx::Conv_715[FLOAT, 32]\n  %onnx::Conv_717[FLOAT, 32x1x3x3]\n  %onnx::Conv_718[FLOAT, 32]\n  %onnx::Conv_720[FLOAT, 16x32x1x1]\n  %onnx::Conv_721[FLOAT, 16]\n  %onnx::Conv_723[FLOAT, 96x16x1x1]\n  %onnx::Conv_724[FLOAT, 96]\n  %onnx::Conv_726[FLOAT, 96x1x3x3]\n  %onnx::Conv_727[FLOAT, 96]\n  %onnx::Conv_729[FLOAT, 24x96x1x1]\n  %onnx::Conv_730[FLOAT, 24]\n  %onnx::Conv_732[FLOAT, 144x24x1x1]\n  %onnx::Conv_733[FLOAT, 144]\n  %onnx::Conv_735[FLOAT, 144x1x3x3]\n  %onnx::Conv_736[FLOAT, 144]\n  %onnx::Conv_738[FLOAT, 24x144x1x1]\n  %onnx::Conv_739[FLOAT, 24]\n  %onnx::Conv_741[FLOAT, 144x24x1x1]\n  %onnx::Conv_742[FLOAT, 144]\n  %onnx::Conv_744[FLOAT, 144x1x3x3]\n  %onnx::Conv_745[FLOAT, 144]\n  %onnx::Conv_747[FLOAT, 32x144x1x1]\n  %onnx::Conv_748[FLOAT, 32]\n  %onnx::Conv_750[FLOAT, 192x32x1x1]\n  %onnx::Conv_751[FLOAT, 192]\n  %onnx::Conv_753

In [ ]:
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow
!pip install -e .

Cloning into 'onnx-tensorflow'...
remote: Enumerating objects: 6516, done.
remote: Counting objects: 100% (465/465), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 6516 (delta 326), reused 383 (delta 261), pack-reused 6051
Receiving objects: 100% (6516/6516), 1.98 MiB | 7.74 MiB/s, done.
Resolving deltas: 100% (5051/5051), done.
Obtaining file:///content/golfdb
ERROR: file:///content/golfdb does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
onnx_model = onnx.load("/content/drive/MyDrive/Hacks/golf/swing.onnx")

In [ ]:
print(onnx.checker.check_model(onnx_model))

None


In [ ]:
!pip install --upgrade onnx-tf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
!onnx-tf convert -i /content/drive/MyDrive/Hacks/golf/swing.onnx -o /content/drive/MyDrive/Hacks/golf/swing.tf

2023-08-25 22:01:55.508790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-08-25 22:01:57,534 - onnx-tf - INFO - Start converting onnx pb to tf saved model
2023-08-25 22:01:58.909852: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Traceback (most recent call last):
  File "/usr/local/bin

In [ ]:
!pwd

/content/golfdb


In [ ]:
%cd /content/golfdb/onnx-tensorflow

/content/golfdb/onnx-tensorflow


In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 9.2 MB/s eta 0:00:00


In [ ]:
from onnx_tf.backend import prepare

tf_rep = prepare(onnx_model)

In [ ]:
tf_rep.export_graph("/content/drive/MyDrive/Hacks/golf/swing.tf")

KeyError: ignored

In [ ]:
# torch.save(model, "/content/drive/MyDrive/Hacks/golf/model_swing.pt")